In [5]:
import pandas as pd
import numpy as np


In [6]:
df= pd.read_csv('Symptom2Disease.csv')

In [7]:
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [8]:
df.columns

Index(['Unnamed: 0', 'label', 'text'], dtype='object')

In [9]:
df=df.drop(columns=['Unnamed: 0'])

In [10]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [12]:
df.shape

(1200, 2)

In [13]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [14]:
df.duplicated()


0       False
1       False
2       False
3       False
4       False
        ...  
1195    False
1196    False
1197    False
1198    False
1199    False
Length: 1200, dtype: bool

In [15]:

nltk.download('stopwords')
nltk.download('wordnet')

NameError: name 'nltk' is not defined

In [16]:
! pip install nltk

In [17]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\advai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\advai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
import re
df['text']=df['text'].str.lower()
df
df['text']=df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df

,label,text
0,Psoriasis,i have been experiencing a skin rash on my arm...
1,Psoriasis,my skin has been peeling especially on my knee...
2,Psoriasis,i have been experiencing joint pain in my fing...
3,Psoriasis,there is a silver like dusting on my skin espe...
4,Psoriasis,my nails have small dents or pits in them and ...
...,...,...
1195,diabetes,im shaking and trembling all over ive lost my ...
1196,diabetes,particularly in the crevices of my skin i have...
1197,diabetes,i regularly experience these intense urges and...
1198,diabetes,i have trouble breathing especially outside i ...


In [19]:
df['text']=df['text'].apply(lambda x: x.split())

In [20]:
print(df['text'][1])

['my', 'skin', 'has', 'been', 'peeling', 'especially', 'on', 'my', 'knees', 'elbows', 'and', 'scalp', 'this', 'peeling', 'is', 'often', 'accompanied', 'by', 'a', 'burning', 'or', 'stinging', 'sensation']


In [21]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\advai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_words = set(stopwords.words('english'))
df['text']=df['text'].apply(lambda x: [word for word in x if word not in stop_words])

In [23]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\advai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
df['processed_text'] = df['text'].apply(lambda x: ' '.join(x))
print(df[['processed_text']].head())


                                      processed_text
0  experiencing skin rash arm leg torso past week...
1  skin peeling especially knee elbow scalp peeli...
2  experiencing joint pain finger wrist knee pain...
3  silver like dusting skin especially lower back...
4  nail small dent pit often feel inflammatory te...


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X = vectorizer.fit_transform(df['processed_text'])

print(X.shape)


(1200, 1374)


In [26]:
df.shape

(1200, 3)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [28]:
X = df['processed_text']
y = df['label']


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [30]:
tfidf = TfidfVectorizer(max_features=5000)  
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [31]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [32]:
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.95
Classification Report:
                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        10
                      Arthritis       1.00      1.00      1.00        10
               Bronchial Asthma       1.00      1.00      1.00        10
           Cervical spondylosis       1.00      1.00      1.00        10
                    Chicken pox       0.82      0.90      0.86        10
                    Common Cold       1.00      1.00      1.00        10
                         Dengue       1.00      0.80      0.89        10
          Dimorphic Hemorrhoids       1.00      1.00      1.00        10
               Fungal infection       1.00      1.00      1.00        10
                   Hypertension       1.00      1.00      1.00        10
                       Impetigo       1.00      1.00      1.00        10
                       Jaundice       1.00      1.00      1.00        10
            

In [33]:
import pickle

with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf, vectorizer_file)


In [34]:

with open('logistic_regression_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    loaded_tfidf = pickle.load(vectorizer_file)


In [35]:
def preprocess_input_text(input_text):
  
    import re

    processed_text = []
    for text in input_text:
        text = text.lower()  
        text = re.sub(r'[^\w\s]', '', text)  
        text = re.sub(r'\d+', '', text)  
        text = re.sub(r'\s+', ' ', text).strip()  
        processed_text.append(text)
    return processed_text


In [38]:
new_text = ["I have increased thirst, frequent urination, fatigue, unexplained weight loss, blurred vision, and slow-healing wounds"]

processed_text = preprocess_input_text(new_text)


new_text_tfidf = loaded_tfidf.transform(processed_text)

predicted_label = loaded_model.predict(new_text_tfidf)

print(f"Predicted Label: {predicted_label[0]}")


Predicted Label: diabetes
